In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv


# 1 Import packages

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits
from tensorflow.keras import layers


# 2 reading file 

In [3]:
data_path="/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv"
lines_raw= pd.read_table(data_path,encoding='utf-8',sep=",")
lines_raw.sample(5)

,source,english_sentence,hindi_sentence
113572,tides,The period 1928-1929 was significant in Indian...,सन् 1928-1929 का दौर भारतीय इतिहास में अत्यंत ...
63857,tides,It was prepared by Roy and released for public...,इसे राय ने तैयार किया था और इस अन्य लोगों द्वा...
96421,tides,The 1963 Act similarly provided for a Hindi tr...,1963 के अधिनियमों में यह भी कहा गया है कि कतिप...
36256,indic2012,External links.,बाहरी कड़ियाँ
2708,tides,The fortuitous bestowing of Man Fridays is she...,अब यह तो संयोग की बात है कि इस फैसले का फायदा ...


In [4]:
lines_raw=lines_raw[lines_raw['source'].isin(['ted'])]
lines_raw

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
...,...,...,...
127595,ted,is if we want that to become our reality -,अगर हम चाहते हैं कि यह वास्तविकता बने -
127597,ted,Africa has not done bad.,अफ़्रीका ने कुछ गलती नहीं की है।
127598,ted,Thank you.,धन्यवाद |
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।


## 3 pre-processing

In [5]:
train_input=lines_raw["english_sentence"]
target_input=lines_raw["hindi_sentence"]

In [6]:
import string
from string import digits

def preprocess_sentence(s):
    s=s.apply(lambda x: x.lower())  ##lower case
    s=s.apply(lambda x: re.sub("'", '', x)) ##remove quote
    s=s.apply(lambda x: ''.join(ch for ch in x if ch not in set(string.punctuation)))  ## remove puncuation
    s=s.apply(lambda x: x.translate(str.maketrans('', '', digits)))
    s=s.apply(lambda x: x.strip())
    s=s.apply(lambda x: re.sub(" +", " ", x))
    try:
        s=s.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
    except:
        pass
    return s

**before preprocessing**

In [7]:
def print_data(data):
    for x in range(10):
        print(x+1, data.iloc[x] )

In [8]:
print_data(train_input )

1 politicians do not have permission to do what needs to be done.
2 I'd like to tell you about one such child,
3 what we really mean is that they're bad at not paying attention.
4 And who are we to say, even, that they are wrong
5 So there is some sort of justice
6 This changed slowly
7 were being produced.
8 And you can see, this LED is going to glow.
9 to turn on the lights or to bring him a glass of water,
10 Can you imagine saying that?


In [9]:
print_data(target_input)

1 राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
2 मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,
3 हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4 और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
5 तो वहाँ न्याय है
6 धीरे धीरे ये सब बदला
7 उत्पन्न नहीं कि जाती थी.
8 और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।
9 लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,
10 क्या आप ये कल्पना कर सकते है


**after preprocessing**

In [10]:
train_input=preprocess_sentence(train_input)
print_data(train_input )

1 politicians do not have permission to do what needs to be done
2 id like to tell you about one such child
3 what we really mean is that theyre bad at not paying attention
4 and who are we to say even that they are wrong
5 so there is some sort of justice
6 this changed slowly
7 were being produced
8 and you can see this led is going to glow
9 to turn on the lights or to bring him a glass of water
10 can you imagine saying that


In [11]:
target_input=preprocess_sentence(target_input)
print_data(target_input)

1 राजनीतिज्ञों के पास जो कार्य करना चाहिए वह करने कि अनुमति नहीं है
2 मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी
3 हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4 और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
5 तो वहाँ न्याय है
6 धीरे धीरे ये सब बदला
7 उत्पन्न नहीं कि जाती थी
8 और जैसा आप देख रहे है ये एलईडी जल उठेगी।
9 लाईट जलाने के लिए या उनके लिए पानी लाने के लिए
10 क्या आप ये कल्पना कर सकते है


**Add "sos, eos"** to target senteces

In [12]:
target_input=target_input.apply(lambda x : '<sos> '+ x+' <eos>')
print_data(target_input)

1 <sos> राजनीतिज्ञों के पास जो कार्य करना चाहिए वह करने कि अनुमति नहीं है <eos>
2 <sos> मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी <eos>
3 <sos> हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते <eos>
4 <sos> और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं <eos>
5 <sos> तो वहाँ न्याय है <eos>
6 <sos> धीरे धीरे ये सब बदला <eos>
7 <sos> उत्पन्न नहीं कि जाती थी <eos>
8 <sos> और जैसा आप देख रहे है ये एलईडी जल उठेगी। <eos>
9 <sos> लाईट जलाने के लिए या उनके लिए पानी लाने के लिए <eos>
10 <sos> क्या आप ये कल्पना कर सकते है <eos>


**tokenize the sentence and OOV**

In [13]:
def token_data(train_preprocessed_input):
    source_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>')
    source_tokenizer.fit_on_texts(train_preprocessed_input)
    source_tokenizer.get_config()
    train_encoder_inputs = source_tokenizer.texts_to_sequences(train_preprocessed_input)
    source_vocab_size = len(source_tokenizer.word_index) + 1
    return train_encoder_inputs,source_vocab_size,source_tokenizer


In [14]:
train_encoder_inputs,source_vocab_size,source_tokenizer = token_data(train_input)
print("source_vocab_size :", source_vocab_size)
train_encoder_inputs[:5]

source_vocab_size : 17347


[[2888, 37, 30, 17, 5251, 4, 37, 22, 483, 4, 28, 200],
 [281, 41, 4, 146, 10, 26, 34, 336, 280],
 [22, 12, 76, 198, 11, 7, 115, 328, 33, 30, 1542, 607],
 [3, 59, 18, 12, 4, 106, 110, 7, 20, 18, 212],
 [15, 38, 11, 83, 376, 5, 1179]]

In [15]:
def generate_decoder_inputs_targets(target_input):
    seqs ,target_vocab_size,target_tokenizer = token_data(target_input)
    decoder_inputs = [s[:-1] for s in seqs] # Drop the last token in the sentence.
    decoder_targets = [s[1:] for s in seqs] # Drop the first token in the sentence.
    return decoder_inputs, decoder_targets,target_vocab_size,target_tokenizer

decoder_inputs, decoder_targets,target_vocab_size,target_tokenizer = generate_decoder_inputs_targets(target_input)
print("target_vocab_size :", target_vocab_size)
decoder_inputs[:5]

target_vocab_size : 22287


[[2, 7408, 6, 80, 22, 316, 76, 139, 65, 46, 12, 1805, 16, 4],
 [2, 1083, 67, 127, 29, 11, 133, 6, 48, 7, 554, 1274],
 [2, 17, 23, 16, 412, 147, 12, 30, 252, 16, 208, 531],
 [2, 5, 17, 138, 416, 8, 14, 678, 25, 122, 12, 71, 248, 8],
 [2, 20, 140, 1522, 4]]

In [16]:
decoder_targets[:5]

[[7408, 6, 80, 22, 316, 76, 139, 65, 46, 12, 1805, 16, 4, 3],
 [1083, 67, 127, 29, 11, 133, 6, 48, 7, 554, 1274, 3],
 [17, 23, 16, 412, 147, 12, 30, 252, 16, 208, 531, 3],
 [5, 17, 138, 416, 8, 14, 678, 25, 122, 12, 71, 248, 8, 3],
 [20, 140, 1522, 4, 3]]

In [17]:
target_tokenizer.word_index["sos"]

2

**Padding**

In [18]:
max_encoding_len = len(max(train_encoder_inputs, key=len))
max_decoding_len = len(max(decoder_inputs, key=len))
print("max_encoding_len",max_encoding_len)
print("max_decoding_len",max_decoding_len)


max_encoding_len 21
max_decoding_len 31


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_train_encoder_inputs = pad_sequences(train_encoder_inputs, max_encoding_len, padding='post', truncating='post')
padded_train_decoder_inputs = pad_sequences(decoder_inputs, max_decoding_len, padding='post', truncating='post')
padded_train_decoder_targets = pad_sequences(decoder_targets, max_decoding_len, padding='post', truncating='post')

In [20]:
padded_train_encoder_inputs[:2]

array([[2888,   37,   30,   17, 5251,    4,   37,   22,  483,    4,   28,
         200,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 281,   41,    4,  146,   10,   26,   34,  336,  280,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [21]:
padded_train_decoder_inputs[:3]

array([[   2, 7408,    6,   80,   22,  316,   76,  139,   65,   46,   12,
        1805,   16,    4,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2, 1083,   67,  127,   29,   11,  133,    6,   48,    7,  554,
        1274,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,   17,   23,   16,  412,  147,   12,   30,  252,   16,  208,
         531,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

# modelling

In [22]:
embedding_dim = 128
hidden_dim = 256
default_dropout=0.2
batch_size = 32
epochs = 30
print("source_vocab_size",source_vocab_size)

source_vocab_size 17347


In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, source_vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = layers.Embedding(source_vocab_size, 
                                          embedding_dim,
                                          name='encoder_embedding_layer')
        self.lstm = layers.LSTM(hidden_dim, 
                                return_sequences=True, 
                                return_state=True,
                                name='encoder_lstm')
    def call(self, input):
        embeddings = self.embedding(input)
        output_seq, state_h, state_c = self.lstm(embeddings)
        return output_seq, state_h, state_c
    
    
    
class LuongAttention(tf.keras.Model):
    def __init__(self, hidden_dim):
        super(LuongAttention, self).__init__()
        self.w = layers.Dense(hidden_dim, name='encoder_outputs_dense')
    def call(self, inputs):
        encoder_output_seq, decoder_output = inputs
        z = self.w(encoder_output_seq)
        attention_scores = tf.matmul(decoder_output, z, transpose_b=True)
        attention_weights = tf.keras.activations.softmax(attention_scores, axis=-1)
        context = tf.matmul(attention_weights, encoder_output_seq)
        return attention_weights, context
    

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self,units):
        super(BahdanauAttention,self).__init__()
        self.Vector=tf.keras.layers.Dense(1)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)

    def call(self,query,value):
        query_with_time_axis = tf.expand_dims(query, 1)
        score=self.Vector(tf.nn.tanh(self.W1(value) +self.W2(query_with_time_axis )))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * value
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()

        self.embedding_layer = layers.Embedding(vocab_size,
                                                embedding_dim,
                                                name='decoder_embedding_layer')

        self.lstm = layers.LSTM(hidden_dim,
                                return_sequences=True,
                                return_state=True,
                                name='decoder_lstm')

        self.attention = LuongAttention(hidden_dim)

        self.w = tf.keras.layers.Dense(hidden_dim, activation='tanh', name='attended_outputs_dense')

        self.dense = layers.Dense(vocab_size, name='decoder_dense')


    def call(self, inputs):
        decoder_input, encoder_output_seq, lstm_state = inputs
        embeddings = self.embedding_layer(decoder_input)

        decoder_output, state_h, state_c = self.lstm(embeddings, initial_state=lstm_state)

        weights, context = self.attention([encoder_output_seq, decoder_output])

        decoder_output_with_attention = self.w(tf.concat(
            [tf.squeeze(context, 1), tf.squeeze(decoder_output, 1)], -1))

        logits = self.dense(decoder_output_with_attention)

        return logits, state_h, state_c, weights
    

    
def loss_func(targets, logits):
        ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)
        return ce_loss(targets, logits, sample_weight=mask)

In [24]:
class TranslatorTrainer(tf.keras.Model):
    def __init__(self, encoder, decoder):
        super(TranslatorTrainer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    @tf.function
    def train_step(self, inputs):
        loss = 0.
        encoder_input_seq, decoder_input_seq, decoder_target_seq = inputs
        with tf.GradientTape() as tape:
            encoder_output_seq, state_h, state_c = self.encoder(encoder_input_seq)
            for i in range(decoder_target_seq.shape[1]):
                next_decoder_input = tf.expand_dims(decoder_input_seq[:, i], 1)
                logits, state_h, state_c, _ = self.decoder(
                  [next_decoder_input, encoder_output_seq, (state_h, state_c)])
                loss += self.loss(decoder_target_seq[:, i], logits)
        variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return {'loss': loss / decoder_target_seq.shape[1]}

In [25]:
dataset = tf.data.Dataset.from_tensor_slices((padded_train_encoder_inputs, 
                                              padded_train_decoder_inputs, 
                                              padded_train_decoder_targets)).batch(batch_size, drop_remainder=True)

**training**

In [26]:
print("source_vocab_size : ",source_vocab_size)
print("embedding_dim     : ", embedding_dim) 
print("hidden_dim        : ", hidden_dim)


source_vocab_size :  17347
embedding_dim     :  128
hidden_dim        :  256


In [27]:
encoder = Encoder(source_vocab_size, embedding_dim, hidden_dim)
decoder = Decoder(target_vocab_size, embedding_dim, hidden_dim)
optimizer = tf.keras.optimizers.Adam()

translator_trainer = TranslatorTrainer(encoder, decoder)
translator_trainer.compile(optimizer=optimizer, loss=loss_func)

In [28]:
epochs = 20
translator_trainer.fit(dataset, epochs=epochs)

Epoch 1/20
1246/1246 [==============================] - 145s 79ms/step - loss: 1.8883
Epoch 2/20
1246/1246 [==============================] - 78s 63ms/step - loss: 1.5601
Epoch 3/20
1246/1246 [==============================] - 78s 63ms/step - loss: 1.3890
Epoch 4/20
1246/1246 [==============================] - 79s 63ms/step - loss: 1.2324
Epoch 5/20
1246/1246 [==============================] - 79s 63ms/step - loss: 1.0904
Epoch 6/20
1246/1246 [==============================] - 79s 63ms/step - loss: 0.9641
Epoch 7/20
1246/1246 [==============================] - 78s 63ms/step - loss: 0.8526
Epoch 8/20
1246/1246 [==============================] - 79s 64ms/step - loss: 0.7565
Epoch 9/20
1246/1246 [==============================] - 78s 63ms/step - loss: 0.6731
Epoch 10/20
1246/1246 [==============================] - 79s 64ms/step - loss: 0.6033
Epoch 11/20
1246/1246 [==============================] - 79s 63ms/step - loss: 0.5447
Epoch 12/20
1246/1246 [==============================] - 80s 6

**prediction**

In [29]:
sentences=(lines_raw.sample(10).iloc[:,1:])
sentences.shape

(10, 2)

In [30]:
sentences.iloc[:1,:]

,english_sentence,hindi_sentence
63111,of whether we could evolve or develop a sixth ...,कि क्या हम एक छठी इन्द्री का विकास या आवर्ण कर...


In [31]:

def Inference(sentences,source_tokenizer,encoder,target_tokenizer, decoder,max_encoding_len,max_decoding_len):
    translations = {'Tokenized Original': [], 'Reference': [], 'Translation': []}
    for s in range(sentences.shape[0]):
        source1, target = sentences["english_sentence"],sentences["hindi_sentence"]
        source2=preprocess_sentence(source1)
        source=source2.iloc[s]
        input_seq = source_tokenizer.texts_to_sequences([source])
        tokenized = source_tokenizer.sequences_to_texts(input_seq)
        input_seq = pad_sequences(input_seq, maxlen=max_encoding_len, padding='post')
        encoder_output, state_h, state_c  = encoder.predict(input_seq)
        current_word = 'sos'
        decoded_sentence = []
        while len(decoded_sentence) < max_decoding_len:
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = target_tokenizer.word_index[current_word]
            logits, state_h, state_c, _ = decoder.predict([target_seq, encoder_output, (state_h, state_c)])
            current_token_index = np.argmax(logits[0])
            current_word = target_tokenizer.index_word[current_token_index]
            if (current_word == 'eos'):
                break
            decoded_sentence.append(current_word)                        
        translations['Tokenized Original'].append(tokenized[0])
        translations['Reference'].append(target.iloc[s])
        translations['Translation'].append( ' '.join(decoded_sentence))
    return translations
   

In [32]:
f=Inference(sentences,source_tokenizer,encoder,target_tokenizer, decoder,max_encoding_len,max_decoding_len)

1/1 [==============================] - 0s 21ms/step


In [33]:
pd.DataFrame(f)

,Tokenized Original,Reference,Translation
0,of whether we could evolve or develop a sixth ...,कि क्या हम एक छठी इन्द्री का विकास या आवर्ण कर...,कि हम एक छठी इन्द्री का विकास या आवर्ण कर सकते...
1,“what ma”,"“क्या, माँ?”",“क्या माँ”
2,how do you make all this work financially,"इस सब को आर्थिक रूप से संभव कैसे बनाया जाय,",कैसे आप सभी इस काम के विषय पर कैसे चलाते हैं
3,you put it on a cooking fire,तुम इसे खाना पकाने की आग पर डाल सकते है.,आप इसे खाना लीजिये ज़मीन में जमा करते हैं।
4,when a group from afghanistan,जब अफगानिस्तान से एक दल,जब अफगानिस्तान में एक दल है
5,is our fear that were not worthy of connection,"है हमारा ये डर कि हम संपर्क के काबिल नहीं हैं,",कि हमारा सबसे ज्यादा संवाद के तौर से नहीं
6,and that is that people want,और वह यह है कि लोग चाहते हैं,और यह है कि लोग चाहते हैं
7,fortunately there are many people like me who ...,"भाग्यवश, मेरे जैसे बहुत से लोग हैं जो कि उपायो...",भाग्यवश मेरे जैसे बहुत से लोग हैं जो कि उपायों...
8,and theres another cognitive bias,"और यहाँ पे एक और दिमागी पक्षपात है,",और वहाँ दिमागी पक्षपात है
9,get the same grades at school,"वही नंबर लाते है, जो कि वो बच्चे",वही नंबर उन्होंने जाना शुरु किया


**Saving model**

In [34]:
# encoder.save_weights('attention_encoder_weights_with_dropout_ckpt')
# decoder.save_weights('attention_decoder_weights_with_dropout_ckpt')

In [35]:
encoder.save_weights('attention_encoder_weights_with_dropout_ckpt')
decoder.save_weights('attention_decoder_weights_with_dropout_ckpt')